In [1]:
!pip install requests

In [2]:
import requests
import json # For pretty printing JSON, though requests.json() handles parsing

# --- Configuration ---
# REPLACE THIS WITH YOUR ACTUAL ALPHA VANTAGE API KEY
API_KEY = "ZRAM7N94Z1U9THS1"
BASE_URL = "https://www.alphavantage.co/query"

# --- Function to fetch real-time (or near real-time) stock data ---
def get_stock_quote(symbol):
    """
    Fetches the latest stock quote for a given ticker symbol from Alpha Vantage.
    """
    params = {
        "function": "GLOBAL_QUOTE", # API function to get a single latest quote
        "symbol": symbol,
        "apikey": API_KEY
    }
    print(f"Fetching data for {symbol}...")
    try:
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)

        data = response.json()

        # Check for API error messages
        if "Error Message" in data:
            print(f"API Error for {symbol}: {data['Error Message']}")
            return None
        if "Note" in data: # Sometimes Alpha Vantage sends notes about API calls/limits
            print(f"API Note for {symbol}: {data['Note']}")
            # Decide if you want to return None or proceed based on the note
            return None # Or proceed, depending on the note's severity

        # Alpha Vantage's GLOBAL_QUOTE data is nested under "Global Quote"
        if "Global Quote" in data and data["Global Quote"]:
            quote = data["Global Quote"]
            # Extract key information. Alpha Vantage keys are a bit messy,
            # they often have numbers like '01. symbol', '05. price'
            # We'll map them to cleaner names.
            stock_info = {
                "Symbol": quote.get("01. symbol"),
                "Open": float(quote.get("02. open", 0)),
                "High": float(quote.get("03. high", 0)),
                "Low": float(quote.get("04. low", 0)),
                "Price": float(quote.get("05. price", 0)),
                "Volume": int(quote.get("06. volume", 0)),
                "Latest_Trading_Day": quote.get("07. latest trading day"),
                "Previous_Close": float(quote.get("08. previous close", 0)),
                "Change": float(quote.get("09. change", 0)),
                "Change_Percent": quote.get("10. change percent") # This is already a string like "1.234%"
            }
            return stock_info
        else:
            print(f"No stock data found for {symbol}. Check the ticker symbol.")
            # print(f"Raw response for debugging: {json.dumps(data, indent=2)}") # Uncomment for debugging
            return None
    except requests.exceptions.RequestException as e:
        print(f"Network or API connection error for {symbol}: {e}")
        return None
    except ValueError as e:
        print(f"Error parsing JSON for {symbol}: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred for {symbol}: {e}")
        return None

# --- Main part of the script ---
if __name__ == "__main__":
    if API_KEY == "YOUR_ALPHA_VANTAGE_API_KEY":
        print("!!! WARNING: Please replace 'YOUR_ALPHA_VANTAGE_API_KEY' with your actual Alpha Vantage API key. !!!")
        print("Get one for free at: https://www.alphavantage.co/support/#api-key")
    else:
        print("\n--- Stock Price Tracker ---")

        # List of ticker symbols to track
        ticker_symbols = ["AAPL", "GOOGL", "MSFT", "TSLA", "AMZN", "NVDA", "IBM", "INVALID_TICKER"]

        for symbol in ticker_symbols:
            stock_data = get_stock_quote(symbol)
            if stock_data:
                print(f"\n--- {stock_data['Symbol']} ---")
                print(f"Current Price: ${stock_data['Price']:.2f}")
                print(f"Previous Close: ${stock_data['Previous_Close']:.2f}")
                print(f"Daily Change: ${stock_data['Change']:.2f} ({stock_data['Change_Percent']})")
                print(f"Volume: {stock_data['Volume']:,}") # Add comma separator for volume
                print(f"Last Trading Day: {stock_data['Latest_Trading_Day']}")
            else:
                print(f"\nCould not retrieve data for {symbol}.")
            print("-" * 30) # Separator


--- Stock Price Tracker ---
Fetching data for AAPL...

--- AAPL ---
Current Price: $202.67
Previous Close: $201.45
Daily Change: $1.22 (0.6056%)
Volume: 54,672,608
Last Trading Day: 2025-06-10
------------------------------
Fetching data for GOOGL...

--- GOOGL ---
Current Price: $178.60
Previous Close: $176.09
Daily Change: $2.51 (1.4254%)
Volume: 61,766,121
Last Trading Day: 2025-06-10
------------------------------
Fetching data for MSFT...

--- MSFT ---
Current Price: $470.92
Previous Close: $472.75
Daily Change: $-1.83 (-0.3871%)
Volume: 15,375,944
Last Trading Day: 2025-06-10
------------------------------
Fetching data for TSLA...

--- TSLA ---
Current Price: $326.09
Previous Close: $308.58
Daily Change: $17.51 (5.6744%)
Volume: 151,256,520
Last Trading Day: 2025-06-10
------------------------------
Fetching data for AMZN...

--- AMZN ---
Current Price: $217.61
Previous Close: $216.98
Daily Change: $0.63 (0.2903%)
Volume: 31,303,317
Last Trading Day: 2025-06-10
----------------